### 1. MCPを使ったエージェント構築
[DatabricksにおけるMCPを用いたエージェントの構築および評価](https://qiita.com/taka_yayoi/items/f2c7ad187ff3acbe0cc6)

次のスニペットを実行して、MCPサーバーへの接続を検証します。このスニペットは、Unity Catalogツールを一覧表示し、その後ベクトル検索インデックスをクエリします。

In [0]:
# %pip install -U --quiet databricks-sdk databricks-langchain databricks-agents databricks-vectorsearch bs4==0.0.2 markdownify==0.14.1 pydantic==2.10.1 databricks-mcp mlflow mcp "databricks-sdk[openai]" databricks-agents
# dbutils.library.restartPython()

In [0]:
%pip install -U databricks-agents databricks-mcp databricks-vectorsearch mlflow
dbutils.library.restartPython()

In [0]:
%run ./00_config

上記のスニペットを基に、ツールを使う基本的なシングルターンエージェントを定義できます。<br>
後続のセクションでデプロイできるよう、エージェントのコードを mcp_agent.py という名前でローカルに保存します。

In [0]:
import asyncio

from mcp.client.streamable_http import streamablehttp_client
from mcp.client.session import ClientSession
from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk import WorkspaceClient

# ワークスペースへの認証を構成
workspace_client = WorkspaceClient()
workspace_hostname = workspace_client.config.host
mcp_server_url = f"{workspace_hostname}/api/2.0/mcp/vector-search/{catalog}/{schema}"

# 以下のスニペットは、Unity Catalog の関数 MCP サーバーを使用して Vector Search Index を公開します
async def test_connect_to_server():
    async with streamablehttp_client(
        f"{mcp_server_url}", auth=DatabricksOAuthClientProvider(workspace_client)
    ) as (read_stream, write_stream, _), ClientSession(
        read_stream, write_stream
    ) as session:
        # MCP サーバーからツールを一覧取得し、呼び出す
        await session.initialize()
        tools = await session.list_tools()
        toolnames = [t.name for t in tools.tools]
        print(
            f"MCP サーバー {mcp_server_url} から検出されたツール: {toolnames}"
        )
        result = await session.call_tool(
            toolnames[0], {"query": "Databricksとは何ですか？"}
        )
        print(
            f"{toolnames[0]} ツールを呼び出し、結果を取得: {result.content}"
        )

await test_connect_to_server()

In [0]:
%%writefile mcp_agent.py

import os
from contextlib import asynccontextmanager
import json
import uuid
import asyncio
from typing import Any, Callable, List
from pydantic import BaseModel
import threading

import mlflow
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse

from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk import WorkspaceClient
from mcp.client.session import ClientSession
from mcp.client.streamable_http import streamablehttp_client

# Databricksノートブック環境でのみnest_asyncioを適用
if os.getenv('DATABRICKS_RUNTIME_VERSION') and 'ipykernel' in os.environ.get('_', ''):
    # Databricksノートブック内
    import nest_asyncio
    nest_asyncio.apply()
    NOTEBOOK_ENV = True
else:
    # Model Servingやその他の環境
    NOTEBOOK_ENV = False

# 1) エンドポイント/プロファイルの設定
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
SYSTEM_PROMPT = "あなたは有能なアシスタントです。"
workspace_client = WorkspaceClient()
host = workspace_client.config.host

# カタログ・データベース名を設定
# catalog = "takaakiyayoi_catalog"
# db = "rag_chatbot_jpn"
import os
catalog = os.environ.get("CATALOG", "default_catalog")
schema = os.environ.get("SCHEMA", "default_schema")

# 必要に応じてMCPサーバーURLを追加
MCP_SERVER_URLS = [
    f"{host}/api/2.0/mcp/vector-search/{catalog}/{schema}",
]

# 2) ResponsesAgent形式の"message dict"をChatCompletions形式に変換するヘルパー
def _to_chat_messages(msg: dict[str, Any]) -> List[dict]:
    """
    ResponsesAgent形式のdictを1つ以上のChatCompletions互換dictに変換
    """
    msg_type = msg.get("type")
    if msg_type == "function_call":
        return [
            {
                "role": "assistant",
                "content": None,
                "tool_calls": [
                    {
                        "id": msg["call_id"],
                        "type": "function",
                        "function": {
                            "name": msg["name"],
                            "arguments": msg["arguments"],
                        },
                    }
                ],
            }
        ]
    elif msg_type == "message" and isinstance(msg["content"], list):
        return [
            {
                "role": "assistant" if msg["role"] == "assistant" else msg["role"],
                "content": content["text"],
            }
            for content in msg["content"]
        ]
    elif msg_type == "function_call_output":
        return [
            {
                "role": "tool",
                "content": msg["output"],
                "tool_call_id": msg["tool_call_id"],
            }
        ]
    else:
        # {"role": ..., "content": "..."}等のプレーンなdictのフォールバック
        return [
            {
                k: v
                for k, v in msg.items()
                if k in ("role", "content", "name", "tool_calls", "tool_call_id")
            }
        ]

# 3) MCPセッションとツール呼び出しロジック
@asynccontextmanager
async def _mcp_session(server_url: str, ws: WorkspaceClient):
    async with streamablehttp_client(
        url=server_url, auth=DatabricksOAuthClientProvider(ws)
    ) as (reader, writer, _):
        async with ClientSession(reader, writer) as session:
            await session.initialize()
            yield session

async def _list_tools_async(server_url: str, ws: WorkspaceClient):
    async with _mcp_session(server_url, ws) as sess:
        return await sess.list_tools()

def _run_async_in_thread(coroutine):
    """
    非同期コルーチンを専用スレッドのイベントループで実行（Model Serving向け）
    """
    result = None
    exception = None
    
    def run_in_thread():
        nonlocal result, exception
        try:
            # 新しいイベントループを作成
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                result = loop.run_until_complete(coroutine)
            finally:
                loop.close()
        except Exception as e:
            exception = e
    
    # 別スレッドで実行
    thread = threading.Thread(target=run_in_thread)
    thread.start()
    thread.join()
    
    if exception:
        raise exception
    return result

def _run_async_safely(coroutine):
    """
    環境に応じて非同期コルーチンを安全に実行
    """
    if NOTEBOOK_ENV:
        # ノートブック: 既存イベントループを利用（nest_asyncio適用済み）
        try:
            loop = asyncio.get_running_loop()
            return asyncio.run(coroutine)
        except RuntimeError:
            # フォールバック: スレッド方式
            return _run_async_in_thread(coroutine)
    else:
        # Model Serving: 常にスレッド方式
        return _run_async_in_thread(coroutine)

def _run_async_in_thread(coroutine):
    """
    非同期コルーチンを専用スレッドのイベントループで実行（Model Serving向け）
    """
    result = None
    exception = None
    
    def run_in_thread():
        nonlocal result, exception
        try:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            try:
                result = loop.run_until_complete(coroutine)
            finally:
                loop.close()
        except Exception as e:
            exception = e
    
    thread = threading.Thread(target=run_in_thread)
    thread.start()
    thread.join()
    
    if exception:
        raise exception
    return result

def _run_async_safely(coroutine):
    """
    環境に応じて非同期コルーチンを安全に実行
    """
    if NOTEBOOK_ENV:
        try:
            loop = asyncio.get_running_loop()
            return asyncio.run(coroutine)
        except RuntimeError:
            return _run_async_in_thread(coroutine)
    else:
        return _run_async_in_thread(coroutine)

def _list_tools(server_url: str, ws: WorkspaceClient):
    # 安全な非同期実行
    return _run_async_safely(_list_tools_async(server_url, ws))

def _make_exec_fn(
    server_url: str, tool_name: str, ws: WorkspaceClient
) -> Callable[..., str]:
    async def call_it_async(**kwargs):
        async with _mcp_session(server_url, ws) as sess:
            resp = await sess.call_tool(name=tool_name, arguments=kwargs)
            return "".join([c.text for c in resp.content])
    
    def exec_fn(**kwargs):
        # 安全な非同期実行
        return _run_async_safely(call_it_async(**kwargs))

    return exec_fn

def _sanitize_tool_name(name: str, max_length: int = 64) -> str:
    """
    Databricks要件に合わせてツール名をサニタイズ
    - 英数字、アンダースコア、ハイフンのみ
    - 最大64文字
    - 正規表現 ^[a-zA-Z0-9_-]{1,64}$ に一致
    """
    import re
    
    # 許可されていない文字をアンダースコアに置換
    sanitized = re.sub(r'[^a-zA-Z0-9_-]', '_', name)
    # 連続アンダースコアを1つに
    sanitized = re.sub(r'_+', '_', sanitized)
    # 先頭・末尾のアンダースコアを除去
    sanitized = sanitized.strip('_')
    # 空なら"tool"に
    if not sanitized:
        sanitized = "tool"
    # 長さ制限
    if len(sanitized) <= max_length:
        result = sanitized
    else:
        # 長すぎる場合は分割して短縮
        if "_" in sanitized:
            parts = sanitized.split("_")
            last_part = parts[-1]
            first_part = parts[0]
            available_chars = max_length - len(last_part) - 1
            if available_chars > 0 and len(first_part) <= available_chars:
                result = f"{first_part}_{last_part}"
            elif available_chars > 0:
                first_part = first_part[:available_chars]
                result = f"{first_part}_{last_part}"
            else:
                result = last_part[:max_length]
        else:
            result = sanitized[:max_length]
    # 最終バリデーション
    pattern = r'^[a-zA-Z0-9_-]{1,64}$'
    if not re.match(pattern, result):
        # 最後の手段: 英数字のみ
        result = re.sub(r'[^a-zA-Z0-9]', '', result)
        if not result:
            result = "tool"
        result = result[:max_length]
    return result

class ToolInfo(BaseModel):
    name: str
    spec: dict
    exec_fn: Callable

def _fetch_tool_infos(ws: WorkspaceClient, server_url: str) -> List[ToolInfo]:
    print(f"MCPサーバー {server_url} からツール一覧を取得")
    infos: List[ToolInfo] = []
    try:
        mcp_tools_result = _list_tools(server_url, ws)
        mcp_tools = mcp_tools_result.tools
        
        for t in mcp_tools:
            # ツール名をサニタイズ
            original_name = t.name
            sanitized_name = _sanitize_tool_name(t.name, 64)
            # バリデーション
            import re
            pattern = r'^[a-zA-Z0-9_-]{1,64}$'
            is_valid = re.match(pattern, sanitized_name)
            print(f"元名: '{original_name}'")
            print(f"サニタイズ後: '{sanitized_name}' (長さ: {len(sanitized_name)}, valid: {bool(is_valid)})")
            if not is_valid:
                print(f"エラー: サニタイズ名がパターンに一致しません!")
                sanitized_name = "vector_search_tool"
            schema = t.inputSchema.copy() if t.inputSchema else {}
            if "properties" not in schema:
                schema["properties"] = {}
            # 説明が長すぎる場合は切り詰め
            description = t.description
            if len(description) > 500:
                description = description[:497] + "..."
            spec = {
                "type": "function",
                "function": {
                    "name": sanitized_name,
                    "description": description,
                    "parameters": schema,
                },
            }
            infos.append(
                ToolInfo(
                    name=original_name,  # 実行時は元名を使う
                    spec=spec,
                    exec_fn=_make_exec_fn(server_url, original_name, ws)
                )
            )
        print(f"{len(infos)}個のツールを正常にロード")
    except Exception as e:
        print(f"{server_url} からのツール取得エラー: {e}")
    return infos

# 4) シングルターン型エージェントクラス
class SingleTurnMCPAgent(ResponsesAgent):
    def __init__(self):
        super().__init__()
        self._tool_infos = None
        self._tools_dict = None
        self._workspace_client = None
        
    def _initialize_tools(self):
        """モデルロード時に一度だけツールを初期化"""
        if self._tool_infos is None:
            try:
                self._workspace_client = WorkspaceClient()
                self._tool_infos = [
                    tool_info
                    for mcp_server_url in MCP_SERVER_URLS
                    for tool_info in _fetch_tool_infos(self._workspace_client, mcp_server_url)
                ]
                self._tools_dict = {tool_info.name: tool_info for tool_info in self._tool_infos}
                print(f"モデルロード時に{len(self._tool_infos)}個のツールを初期化")
            except Exception as e:
                print(f"警告: モデルロード時のツール初期化失敗: {e}")
                self._tool_infos = []
                self._tools_dict = {}
    
    def _call_llm(self, history: List[dict], ws: WorkspaceClient, tool_infos):
        """
        現在の履歴をLLMに送信し、生のレスポンスdictを返す
        """
        client = ws.serving_endpoints.get_open_ai_client()
        flat_msgs = []
        for msg in history:
            flat_msgs.extend(_to_chat_messages(msg))

        # Databricksツール形式に変換
        tools_param = None
        if tool_infos:
            tools_param = []
            for ti in tool_infos:
                function_spec = ti.spec["function"]
                tool_dict = {
                    "type": "function",
                    "function": {
                        "name": function_spec["name"],
                        "description": function_spec["description"]
                    }
                }
                # パラメータが存在し空でなければ追加
                if function_spec.get("parameters") and function_spec["parameters"].get("properties"):
                    tool_dict["function"]["parameters"] = function_spec["parameters"]
                else:
                    # 空のパラメータ仕様
                    tool_dict["function"]["parameters"] = {
                        "type": "object",
                        "properties": {}
                    }
                tools_param.append(tool_dict)
            # ノートブック環境のみデバッグ出力
            if NOTEBOOK_ENV:
                print(f"LLMに{len(tools_param)}個のツールを送信")
                for i, tool in enumerate(tools_param):
                    print(f"ツール {i}: {tool['function']['name']}")
                    import json
                    print(f"ツール構造: {json.dumps(tool, indent=2)}")

        # 複数アプローチで実行
        try:
            # まずtoolsパラメータ付きで実行
            if tools_param:
                return client.chat.completions.create(
                    model=LLM_ENDPOINT_NAME,
                    messages=flat_msgs,
                    tools=tools_param,
                )
            else:
                return client.chat.completions.create(
                    model=LLM_ENDPOINT_NAME,
                    messages=flat_msgs,
                )
        except Exception as e:
            if NOTEBOOK_ENV:
                print(f"最初の試行失敗: {e}")
                print("ツールなしでフォールバック...")
            # フォールバック: ツールなしで実行
            return client.chat.completions.create(
                model=LLM_ENDPOINT_NAME,
                messages=flat_msgs,
            )

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        # 未初期化ならツールを初期化
        self._initialize_tools()
        
        ws = self._workspace_client or WorkspaceClient()

        # 1) system+userで初期履歴を構築
        history: List[dict] = [{"role": "system", "content": SYSTEM_PROMPT}]
        for inp in request.input:
            history.append(inp.model_dump())

        # 2) LLMを一度呼び出し
        try:
            # 事前ロード済みツールを利用
            tool_infos = self._tool_infos
            tools_dict = self._tools_dict
            
            if NOTEBOOK_ENV:
                print(f"事前ロード済みツール数: {len(tool_infos)}")
            
            llm_resp = self._call_llm(history, ws, tool_infos)
            raw_choice = llm_resp.choices[0].message.to_dict()
            raw_choice["id"] = uuid.uuid4().hex
            history.append(raw_choice)

            tool_calls = raw_choice.get("tool_calls") or []
            if tool_calls:
                # （この例では単一ツールのみ対応）
                fc = tool_calls[0]
                requested_name = fc["function"]["name"]
                args = json.loads(fc["function"]["arguments"])
                # サニタイズ名から元名を検索
                original_name = None
                for tool_info in tool_infos:
                    if tool_info.spec["function"]["name"] == requested_name:
                        original_name = tool_info.name
                        break
                if original_name and original_name in tools_dict:
                    try:
                        tool_info = tools_dict[original_name]
                        result = tool_info.exec_fn(**args)
                    except Exception as e:
                        result = f"{original_name}の呼び出しエラー: {e}"
                else:
                    result = f"ツール {requested_name} が見つかりません"
                # 4) "tool"出力を履歴に追加
                history.append(
                    {
                        "type": "function_call_output",
                        "role": "tool",
                        "id": uuid.uuid4().hex,
                        "tool_call_id": fc["id"],
                        "output": result,
                    }
                )
                # 5) LLMを再度呼び出し、その返答を最終とする
                followup = (
                    self._call_llm(history, ws, tool_infos=[]).choices[0].message.to_dict()
                )
                followup["id"] = uuid.uuid4().hex

                assistant_text = followup.get("content", "")
                return ResponsesAgentResponse(
                    output=[
                        {
                            "id": uuid.uuid4().hex,
                            "type": "message",
                            "role": "assistant",
                            "content": [{"type": "output_text", "text": assistant_text}],
                        }
                    ],
                    custom_outputs=request.custom_inputs,
                )

            # 6) tool_callsがなければ元のassistant返答を返す
            assistant_text = raw_choice.get("content", "")
            return ResponsesAgentResponse(
                output=[
                    {
                        "id": uuid.uuid4().hex,
                        "type": "message",
                        "role": "assistant",
                        "content": [{"type": "output_text", "text": assistant_text}],
                    }
                ],
                custom_outputs=request.custom_inputs,
            )
        
        except Exception as e:
            # エラー処理
            error_message = f"リクエスト処理中のエラー: {str(e)}"
            print(error_message)
            return ResponsesAgentResponse(
                output=[
                    {
                        "id": uuid.uuid4().hex,
                        "type": "message",
                        "role": "assistant",
                        "content": [{"type": "output_text", "text": error_message}],
                    }
                ],
                custom_outputs=request.custom_inputs,
            )

# MLflowモデルをセット
mlflow.models.set_model(SingleTurnMCPAgent())

# テスト実行
try:
    print("エージェントリクエスト作成中...")
    req = ResponsesAgentRequest(
        input=[{"role": "user", "content": "Databricksとは？"}]
    )
    
    print("予測実行中...")
    agent = SingleTurnMCPAgent()
    resp = agent.predict(req)
    
    print("レスポンス:")
    for item in resp.output:
        print(item)
        
except Exception as e:
    print(f"実行中のエラー: {e}")
    import traceback
    traceback.print_exc()

### 2. MCPを使用してエージェントをデプロイする

MCPサーバーに接続するエージェントをデプロイする準備ができたら、[標準のエージェントデプロイメントプロセス](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/deploy-agent)を使用してください。<br>

[エージェントがアクセスする必要があるすべてのリソースをログイン時に指定する](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/log-agent#authentication-for-databricks-resources)ことを確認してください。<br>
例えば、エージェントが以下のMCPサーバーURLを使用する場合：<br>
`https://<your-workspace-hostname>/api/2.0/mcp/vector-search/prod/customer_support`<br>
`https://<your-workspace-hostname>/api/2.0/mcp/vector-search/prod/billing`<br>
`https://<your-workspace-hostname>/api/2.0/mcp/functions/prod/billing`<br>
エージェントが必要とするすべてのベクトル検索インデックス、およびすべてのUnity Catalog関数をリソースとして指定する必要があります。<br>

エージェントが必要とするすべてのベクトル検索インデックス、およびすべてのUnity Catalog関数をリソースとして指定する必要があります。<br>
例えば、上記で定義されたエージェントをデプロイするには、エージェントコード定義をmcp_agent.pyに保存したと仮定して、次のスニペットを実行できます。<br>
Pythonカーネルを再起動してimportするファイルを認識できるようにします。


In [0]:
%restart_python

In [0]:
%run ./00_config

In [0]:
# 環境変数を設定
import os
os.environ["CATALOG"] = catalog
os.environ["SCHEMA"] = schema

In [0]:
import os
from databricks.sdk import WorkspaceClient
from databricks import agents
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint, DatabricksVectorSearchIndex
from mcp_agent import LLM_ENDPOINT_NAME

workspace_client = WorkspaceClient()

# mcp_agent.pyで定義されたエージェントをログ
agent_script = "mcp_agent.py"
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    # --- エージェントコード内のMCP_SERVER_URLSを介して参照される場合、以下の行をアンコメントしてベクトル検索インデックスや追加のUC関数を指定 ---
    DatabricksVectorSearchIndex(index_name=f"{catalog}.{schema}.gold_feedbacks_index"),
    # DatabricksVectorSearchIndex(index_name="prod.billing.another_index"),
    # DatabricksFunction(f"{catalog}.{schema}.get_store_product_inventory"),
    # DatabricksFunction(f"{catalog}.{schema}.get_store_item_sales_ranking"),
    # DatabricksFunction(f"{catalog}.{schema}.get_store_sales_ranking"),
]

with mlflow.start_run():
    logged_model_info = mlflow.pyfunc.log_model(
        name="mcp_agent",
        python_model=agent_script,
        resources=resources,
    )

# TODO UCモデル名をここに指定
UC_MODEL_NAME = f"{catalog}.{schema}.bricksmart_analysis_mcp_agent"
registered_model = mlflow.register_model(logged_model_info.model_uri, UC_MODEL_NAME)

deployment_info = agents.deploy(
    model_name=UC_MODEL_NAME,
    model_version=registered_model.version,
)


In [0]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
for model_name in [f"{catalog}.{schema}.bricksmart_analysis_mcp_agent", f"{catalog}.{schema}.feedback"]:
    try:
        client.get_registered_model(model_name)
        client.delete_registered_model(model_name)
    except Exception:
        pass  # モデルがなければ何もしない